In [275]:
import json
import pandas as pd
from pymongo import MongoClient

with open(r"C:\Users\Ananya P Salunkhe\Downloads\TakeHome-Data Scientist\Data\product_info.json", "r") as file:
    raw_data = json.load(file)

product_data = raw_data["data"]
df= pd.DataFrame(product_data)

df.head()

,index,Product ID,Category,Sub-Category,Product Name
0,0,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase
1,1,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,..."
2,2,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...
3,3,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table
4,4,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System


In [276]:
df.drop(columns=["index"], inplace=True, errors="ignore")

df.columns = df.columns.str.lower().str.replace(" ","_")

In [277]:
df.shape

(1894, 4)

In [278]:
df.isnull().sum()

product_id      0
category        0
sub-category    0
product_name    0
dtype: int64

No null values/missing values

In [279]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_id    1894 non-null   object
 1   category      1894 non-null   object
 2   sub-category  1894 non-null   object
 3   product_name  1894 non-null   object
dtypes: object(4)
memory usage: 59.3+ KB


In [280]:
df.nunique()

product_id      1862
category           3
sub-category      17
product_name    1850
dtype: int64

→ Some product IDs are repeated (1,894 total rows vs. 1,862 unique)
→ This suggests that some products might be duplicated or have variations

→ There are 1,850 unique product names but 1,894 total rows.
→ Some products might have the same name but different IDs, or duplicates exist.

In [281]:
df.describe()

,product_id,category,sub-category,product_name
count,1894,1894,1894,1894
unique,1862,3,17,1850
top,TEC-PH-10002200,Office Supplies,Paper,Staples
freq,2,1098,284,10


In [282]:
duplicate = df[df.duplicated()]
duplicate

,product_id,category,sub-category,product_name


No completely duplicate rows

In [283]:
# sorted filtered df that contains rows where product_id is duplicated
duplicate_ids = df[df.duplicated(subset="product_id", keep=False)]
duplicate_ids.sort_values(by="product_id")

,product_id,category,sub-category,product_name
1364,FUR-BO-10002213,Furniture,Bookcases,"Sauder Forest Hills Library, Woodland Oak Finish"
1259,FUR-BO-10002213,Furniture,Bookcases,DMI Eclipse Executive Suite Bookcases
64,FUR-CH-10001146,Furniture,Chairs,"Global Value Mid-Back Manager's Chair, Gray"
124,FUR-CH-10001146,Furniture,Chairs,"Global Task Chair, Black"
1010,FUR-FU-10001473,Furniture,Furnishings,DAX Wood Document Frame
...,...,...,...,...
890,TEC-PH-10002200,Technology,Phones,Samsung Galaxy Note 2
1399,TEC-PH-10002310,Technology,Phones,Plantronics Calisto P620-M USB Wireless Speake...
974,TEC-PH-10002310,Technology,Phones,Panasonic KX T7731-B Digital phone
727,TEC-PH-10004531,Technology,Phones,OtterBox Commuter Series Case - iPhone 5 & 5s


In [284]:
# counting no of duplicates for duplicate product ids
df["product_id"].value_counts()[df["product_id"].value_counts() > 1]

product_id
TEC-PH-10002200    2
FUR-FU-10004864    2
OFF-BI-10002026    2
TEC-AC-10002049    2
FUR-FU-10004848    2
TEC-PH-10001530    2
OFF-ST-10001228    2
TEC-PH-10001795    2
OFF-PA-10001166    2
OFF-PA-10000659    2
FUR-FU-10004017    2
TEC-AC-10003832    2
OFF-PA-10001970    2
OFF-PA-10000357    2
OFF-PA-10002377    2
TEC-PH-10004531    2
OFF-BI-10004632    2
FUR-CH-10001146    2
OFF-PA-10002195    2
OFF-AR-10001149    2
OFF-AP-10000576    2
OFF-BI-10004654    2
TEC-MA-10001148    2
FUR-FU-10004091    2
TEC-PH-10002310    2
FUR-FU-10001473    2
FUR-FU-10004270    2
OFF-ST-10004950    2
OFF-PA-10000477    2
FUR-BO-10002213    2
TEC-AC-10002550    2
OFF-PA-10003022    2
Name: count, dtype: int64

In [285]:
# counting no of duplicates for duplicate product ids
df["product_id"].value_counts()[df["product_id"].value_counts() > 2]

Series([], Name: count, dtype: int64)

Meaning max 2 products have same product_id

In [286]:
# all duplicate product names
df[df.duplicated(subset="product_name", keep=False)].sort_values("product_name")

,product_id,category,sub-category,product_name
760,OFF-EN-10000461,Office Supplies,Envelopes,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes"
1224,OFF-EN-10000781,Office Supplies,Envelopes,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes"
1382,OFF-BI-10000829,Office Supplies,Binders,Avery Non-Stick Binders
308,OFF-BI-10004140,Office Supplies,Binders,Avery Non-Stick Binders
33,OFF-PA-10000249,Office Supplies,Paper,Easy-staple paper
819,OFF-PA-10003127,Office Supplies,Paper,Easy-staple paper
892,OFF-PA-10001685,Office Supplies,Paper,Easy-staple paper
1741,OFF-PA-10002764,Office Supplies,Paper,Easy-staple paper
817,OFF-PA-10000349,Office Supplies,Paper,Easy-staple paper
259,OFF-PA-10000474,Office Supplies,Paper,Easy-staple paper


In [287]:
# count for same product names for different product ids
df["product_name"].value_counts()[df["product_name"].value_counts() > 1]

product_name
Staples                                          10
Staple envelope                                   9
Easy-staple paper                                 8
Staples in misc. colors                           7
Staple holder                                     3
Staple remover                                    3
#10- 4 1/8" x 9 1/2" Recycled Envelopes           2
Eldon Wave Desk Accessories                       2
KI Adjustable-Height Table                        2
Staple-based wall hangings                        2
Storex Dura Pro Binders                           2
Staple magnet                                     2
Avery Non-Stick Binders                           2
Prang Drawing Pencil Set                          2
Peel & Seel Recycled Catalog Envelopes, Brown     2
Okidata C610n Printer                             2
Name: count, dtype: int64

→ Duplicate product name for different product_id is not an issue as the product_id is the primary key and it remains unique

In [288]:
from collections import defaultdict

# Dictionary to track occurrences of each product_id
existing_counts = defaultdict(int)

# appending digit for duplicates
def make_unique(pid):
    if existing_counts[pid] > 0:  
        new_id = f"{pid}{existing_counts[pid]}"  
    else:
        new_id = pid  # first occurrence unchanged
    
    existing_counts[pid] += 1  # Increment count for future duplicates
    return new_id

df["product_id"] = df["product_id"].apply(make_unique)

In [289]:
# Check
df[df["product_id"].str.contains("FUR-BO-10002213", na=False)]

,product_id,category,sub-category,product_name
1259,FUR-BO-10002213,Furniture,Bookcases,DMI Eclipse Executive Suite Bookcases
1364,FUR-BO-100022131,Furniture,Bookcases,"Sauder Forest Hills Library, Woodland Oak Finish"


Now all duplicate product ids have become unique by appending 1,2,3...

→ In our case a product id is duplicated atmost 2 times, hence the first occurence remains same, and 1 is appended to the second occurence

In [290]:
df.to_excel("product_info.xlsx", index = False)